In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist


In [ ]:
%matplotlib inline

In [ ]:
'''Load and save functions.'''

filepath = r'bbb'

# load file
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    return pd.read_csv(f)

# save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '/' + "{}".format(fName), sep =',', index=False)
    return print("Saved {} to {}".format(fName,filepath))

# load GTFS files
trips = load('trips')
stops = load('stops')
stop_times = load('stop_times')
shapes = load('shapes')



In [ ]:
'''USED FOR TESTING- query a single trip, shape, stops'''
# # get one trip to test 
# onetrip = trips[(trips.trip_id == 782006)]
# oneshape = shapes[(shapes.shape_id == 23897)]
# stopTimeStops = stop_times['stop_id'][(stop_times.trip_id == 782006)].tolist()
# # stopTimeStops = (stop_times.trip_id == 782006).tolist()
# neededStops = stops[stops['stop_id'].isin(stopTimeStops)]
# lstlat = neededStops['stop_lat'].tolist()
# lstlon = neededStops['stop_lon'].tolist()
# stpCoord = np.array(list(zip(lstlat,lstlon)))
# # stopcoordslst = list(result)

In [153]:
shapes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21860 entries, 0 to 21859
Data columns (total 5 columns):
shape_id               21860 non-null int64
shape_pt_lat           21860 non-null float64
shape_pt_lon           21860 non-null float64
shape_pt_sequence      21860 non-null int64
shape_dist_traveled    21860 non-null float64
dtypes: float64(3), int64(2)
memory usage: 854.0 KB


In [155]:
'''MAIN WORK HERE'''

# turn stops coords to list, then numpy array
# this will be used for the distance function
stpLat = stops['stop_lat'].tolist()
stpLon = stops['stop_lon'].tolist()
stpCoord = np.array(list(zip(stpLat,stpLon)))

# turn shape coords to list, then turn to numpy array
# this will be used for the distance function
shpLat = shapes['shape_pt_lat'].tolist()
shpLon = shapes['shape_pt_lon'].tolist()
shpCoord = np.array(list(zip(shpLat,shpLon)))

# create a dataframe to use as a way to identify the rows that need to be changed 
nearest_shapePnt_df = pd.DataFrame({
    'shape_id' : shapes.shape_id,
    'shape_pt_lat' : shapes.shape_pt_lat,
    'shape_pt_lon' : shapes.shape_pt_lon,
    'shape_pt_sequence' : shapes.shape_pt_sequence,
    'shape_dist_traveled' : shapes.shape_dist_traveled,
    # use the euclidean distance function here and multiply by 10000 to get some eaiser numbers to work with
    'dist_to_stp': cdist(shpCoord,stpCoord,'euclidean').min(axis=1) * 10000

})

''''''

# find difference between dist_to_stp rows
nearest_shapePnt_df['diff'] = nearest_shapePnt_df['dist_to_stp'] - nearest_shapePnt_df['dist_to_stp'].shift(1)
# create a dummy column marking where the diff is negative
nearest_shapePnt_df['dummy'] = np.where(nearest_shapePnt_df['diff'] < 0,'1','0')
'''Keep where diff == NaN- this is first shape point and 'should' begin at a stop OR where distance to stop is < 2.8 OR where dummy == 1 AND the the value below it is == 0
AND Keep where dummy == 1 AND distance to stop is < 5 (this may change). This logic will ensure the closest stop is grabbed based on criteria other than spatial criteria. 
The result is a column named "keep" with values of either keep or throw. These values are used later on with a buffer from the stop to highlight the rows to change'''
nearest_shapePnt_df['keep'] = np.where(np.logical_or(nearest_shapePnt_df['diff'].isna(),\
                                                     np.logical_or(nearest_shapePnt_df['dist_to_stp'] < 2.8,\
                                                                    np.logical_and(np.logical_and(nearest_shapePnt_df['dummy'] == '1', nearest_shapePnt_df['dist_to_stp'] < 5.),\
                                                                                   np.logical_and(nearest_shapePnt_df['dummy'] == '1', nearest_shapePnt_df['dummy'].shift(-1) == '0')\
                                                                                  )\
                                                                  )\
                                                    ) ,'keep','throw')


'''With all of the other parameters above, have a column with a buffer of the stop and check to see if a shape point marked as "keep"
falls within that buffer. If it does, update the shape point lat and lon for that row. There may be more than one shape point moved, inital testng 
has shown this not to be a problem. The generaliztion around the stops "should" not be noticeable. More testing should be done to 100% confirm'''

# create a geodataframe to do an intersection 
intersect_df = gpd.GeoDataFrame(nearest_shapePnt_df, geometry = gpd.points_from_xy(nearest_shapePnt_df.shape_pt_lon,nearest_shapePnt_df.shape_pt_lat))
intersect_df['shape_pt_geometry'] = intersect_df['geometry']
intersect_df.drop(['geometry'],axis=1)

# stop dataframe with a buffer- distance can be adjusted if need be
stopdf = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon,stops.stop_lat))
stopdf['geometry'] = testStopdf.geometry.buffer(.0005)

# intersect df based on buffer polygon
intersect_join = gpd.sjoin(intersect_df,stopdf,how='inner',op='intersects')
# only keep the records 'keep'
intersect_join = intersect_join[(intersect_join.keep == 'keep')]

'''join the intersect df back to the nearest shape df based on index and create the new shapes.txt'''
badMerge = pd.merge(nearest_shapePnt_df,intersect_join,how='left',suffixes=('','_y'),left_index=True,right_index=True)
# got a duplicated index somehow, drop it here
finalMerge = badMerge[~badMerge.index.duplicated()]

# update the new join shape lat lon with stop lat lon based on the 'keep' column
finalMerge.loc[finalMerge['keep'] == 'keep', 'shape_pt_lat'] = finalMerge['stop_lat']
finalMerge.loc[finalMerge['keep'] == 'keep', 'shape_pt_lon'] = finalMerge['stop_lon']


# drop unwanted columns from the final merge df to create the shapes_NEW.txt
dropCols = [i for i in range(len(finalMerge.columns)) if i > 4]
finalMerge.drop(finalMerge.columns[dropCols],axis=1,inplace=True)
# finalMerge.info()

#  print the amount of points that were changed
changedPoints = finalMerge.shape_pt_lat != shapes.shape_pt_lat
print('SWEET! You modified {} shape points.'.format(len(finalMerge[changedPoints])))

# finalMerge.head()

# this is the final product- OLD WAY, keep this for testing
# finalMerge[['shape_id','shape_pt_lat','shape_pt_lon', 'shape_pt_sequence']]
# finalMerge.info()


SWEET! You modified 5880 shape points
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21860 entries, 0 to 21859
Data columns (total 5 columns):
shape_id               21860 non-null int64
shape_pt_lat           21860 non-null float64
shape_pt_lon           21860 non-null float64
shape_pt_sequence      21860 non-null int64
shape_dist_traveled    21860 non-null float64
dtypes: float64(3), int64(2)
memory usage: 1.6 MB


In [156]:
finalMerge.head()


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,23840,33.988695,-118.471527,1,0.0000
1,23840,33.988695,-118.471527,2,0.0480
2,23840,33.988920,-118.471769,3,0.0769
3,23840,33.989380,-118.472100,4,0.1360
4,23840,33.989740,-118.472389,5,0.1837


In [ ]:
'''Test graph to look at the distances of the shape points in comparison to stops along the route'''
# plot the distance from the stop... the 'valleys' are where the stops are
# plt.figure()
# plt.ylabel("Distance From Stop").set_color("White")
# nearest_shapePnt_df['dist_to_stp'].plot(kind='bar',figsize=(20,7)).axes.get_xaxis().set_visible(False)
# plt.show()

In [ ]:
'''Test map (graph) to look at ALL of the shape points for a route in comparison to stops along the route'''
# plot the stops (red) and the shape points the script returns (blue)
# all of the shape points FOR TESTING
# allShapePoints = gpd.GeoDataFrame(oneshape, geometry=gpd.points_from_xy(oneshape.shape_pt_lon,oneshape.shape_pt_lat))
# stopsTest = testStopdf.plot(figsize=(15,15),color='red', marker='o',markersize=150)
# # allShapePoints.plot(figsize=(15,15),color='blue')
# allShapePoints.plot(ax=stopsTest, color='blue',marker='o',markersize=20)
# plt.show()




In [ ]:
'''Test map (graph) to look at a subset of the shape points identifed as being nearest to stops along the route'''
# # testPointsdf = gpd.GeoDataFrame(oneshape, geometry=gpd.points_from_xy(oneshape.shape_pt_lon,oneshape.shape_pt_lat))
# testPointsdf = gpd.GeoDataFrame(keepPnts, geometry=gpd.points_from_xy(keepPnts.shape_pt_lon,keepPnts.shape_pt_lat))
# testPointsdf
# testStopdf = gpd.GeoDataFrame(neededStops, geometry=gpd.points_from_xy(neededStops.stop_lon,neededStops.stop_lat))
# testStopdf['geometry'] = testStopdf.geometry.buffer(0.0010)
# testStopdf
#plot shape points and stops... see an extra shape point near southern part of route
# testPointsdf = gpd.GeoDataFrame(keepPnts, geometry=gpd.points_from_xy(keepPnts.shape_pt_lon,keepPnts.shape_pt_lat))
# newShapesGdf = gpd.GeoDataFrame(newShapesTxt, geometry=gpd.points_from_xy(newShapesTxt.shape_pt_lon,newShapesTxt.shape_pt_lat))
# stopsTest = testStopdf.plot(figsize=(15,15),color='red',marker='o',markersize=150)
# newShapesGdf.plot(ax=stopsTest, color='blue',marker='o',markersize=20)
# plt.show()
    